<a href="https://colab.research.google.com/github/jyotidabass/Traditional-feature-extraction-methods/blob/main/Traditional_feature_extraction_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#4 texture features are extracted from this

import cv2
import numpy as np
import os
import glob
import time

tic = time.time()

def coarseness(image, kmax):
	image = np.array(image)
	w = image.shape[0]
	h = image.shape[1]
	kmax = kmax if (np.power(2,kmax) < w) else int(np.log(w) / np.log(2))
	kmax = kmax if (np.power(2,kmax) < h) else int(np.log(h) / np.log(2))
	average_gray = np.zeros([kmax,w,h])
	horizon = np.zeros([kmax,w,h])
	vertical = np.zeros([kmax,w,h])
	Sbest = np.zeros([w,h])

	for k in range(kmax):
		window = np.power(2,k)
		for wi in range(w)[window:(w-window)]:
			for hi in range(h)[window:(h-window)]:
				average_gray[k][wi][hi] = np.sum(image[wi-window:wi+window, hi-window:hi+window])
		for wi in range(w)[window:(w-window-1)]:
			for hi in range(h)[window:(h-window-1)]:
				horizon[k][wi][hi] = average_gray[k][wi+window][hi] - average_gray[k][wi-window][hi]
				vertical[k][wi][hi] = average_gray[k][wi][hi+window] - average_gray[k][wi][hi-window]
		horizon[k] = horizon[k] * (1.0 / np.power(2, 2*(k+1)))
		vertical[k] = horizon[k] * (1.0 / np.power(2, 2*(k+1)))

	for wi in range(w):
		for hi in range(h):
			h_max = np.max(horizon[:,wi,hi])
			h_max_index = np.argmax(horizon[:,wi,hi])
			v_max = np.max(vertical[:,wi,hi])
			v_max_index = np.argmax(vertical[:,wi,hi])
			index = h_max_index if (h_max > v_max) else v_max_index
			Sbest[wi][hi] = np.power(2,index)

	fcrs = np.mean(Sbest)
	return fcrs


def contrast(image):
	image = np.array(image)
	image = np.reshape(image, (1, image.shape[0]*image.shape[1]))
	m4 = np.mean(np.power(image - np.mean(image),4))
	v = np.var(image)
	std = np.power(v, 0.5)
	alfa4 = m4 / np.power(v,2)
	fcon = std / np.power(alfa4, 0.25)
	return fcon

def directionality(image):
	image = np.array(image, dtype = 'int64')
	h = image.shape[0]
	w = image.shape[1]
	convH = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
	convV = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
	deltaH = np.zeros([h,w])
	deltaV = np.zeros([h,w])
	theta = np.zeros([h,w])

	# calc for deltaH
	for hi in range(h)[1:h-1]:
		for wi in range(w)[1:w-1]:
			deltaH[hi][wi] = np.sum(np.multiply(image[hi-1:hi+2, wi-1:wi+2], convH))
	for wi in range(w)[1:w-1]:
		deltaH[0][wi] = image[0][wi+1] - image[0][wi]
		deltaH[h-1][wi] = image[h-1][wi+1] - image[h-1][wi]
	for hi in range(h):
		deltaH[hi][0] = image[hi][1] - image[hi][0]
		deltaH[hi][w-1] = image[hi][w-1] - image[hi][w-2]

	# calc for deltaV
	for hi in range(h)[1:h-1]:
		for wi in range(w)[1:w-1]:
			deltaV[hi][wi] = np.sum(np.multiply(image[hi-1:hi+2, wi-1:wi+2], convV))
	for wi in range(w):
		deltaV[0][wi] = image[1][wi] - image[0][wi]
		deltaV[h-1][wi] = image[h-1][wi] - image[h-2][wi]
	for hi in range(h)[1:h-1]:
		deltaV[hi][0] = image[hi+1][0] - image[hi][0]
		deltaV[hi][w-1] = image[hi+1][w-1] - image[hi][w-1]

	deltaG = (np.absolute(deltaH) + np.absolute(deltaV)) / 2.0
	deltaG_vec = np.reshape(deltaG, (deltaG.shape[0] * deltaG.shape[1]))

	# calc the theta
	for hi in range(h):
		for wi in range(w):
			if (deltaH[hi][wi] == 0 and deltaV[hi][wi] == 0):
				theta[hi][wi] = 0;
			elif(deltaH[hi][wi] == 0):
				theta[hi][wi] = np.pi
			else:
				theta[hi][wi] = np.arctan(deltaV[hi][wi] / deltaH[hi][wi]) + np.pi / 2.0
	theta_vec = np.reshape(theta, (theta.shape[0] * theta.shape[1]))

	n = 16
	t = 12
	cnt = 0
	hd = np.zeros(n)
	dlen = deltaG_vec.shape[0]
	for ni in range(n):
		for k in range(dlen):
			if((deltaG_vec[k] >= t) and (theta_vec[k] >= (2*ni-1) * np.pi / (2 * n)) and (theta_vec[k] < (2*ni+1) * np.pi / (2 * n))):
				hd[ni] += 1
	hd = hd / np.mean(hd)
	hd_max_index = np.argmax(hd)
	fdir = 0
	for ni in range(n):
		fdir += np.power((ni - hd_max_index), 2) * hd[ni]
	return fdir

def roughness(fcrs, fcon):
	return fcrs + fcon

if __name__ == '__main__':
        
        # load the training dataset
        train_path  = "/content/drive/MyDrive/TWO-CLASS COATED-UNCOATED/Train/Coated tongue"
        train_names = os.listdir(train_path)

        # loop over the training dataset
        cur_path = os.path.join(train_path, '*g')
        cur_label = train_names
        i = 0

        for file in glob.glob(cur_path):
            print('For image {} named {}:'.format(i+1,cur_label[i]))
            img = cv2.imread(file)
            img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            print('Shape of image is: {} '.format(img.shape))
            fcrs = coarseness(img, 5)
            print("coarseness: %f" % fcrs);
            fcon = contrast(img)
            print("contrast: %f" % fcon)
            fdir= directionality(img)
            print("directionality: %f" % fdir)
            f_r=roughness(fcrs,fcon)
            print("roughness: %f" % f_r)
            print('\n\n')
            i+=1

toc = time.time()
print("Computation time is {} minutes".format((toc-tic)/60))

For image 1 named 0CA41A93-47F3-40CD-9479-8FB4CCFED190 - vinay kumar.jpeg:
Shape of image is: (1291, 1019) 
coarseness: 10.012288
contrast: 34.766788
directionality: 536.730510
roughness: 44.779076



For image 2 named 5E2724F4-504A-4820-8328-EBADDD42AE00 - G D sivakumar.jpeg:
Shape of image is: (1086, 1294) 
coarseness: 10.471606
contrast: 24.352150
directionality: 580.965649
roughness: 34.823756



For image 3 named 8B58C21B-2441-480A-A1F1-2043488FF2AA - Sharu Mathi.jpeg:
Shape of image is: (1366, 1215) 
coarseness: 10.273066
contrast: 39.123193
directionality: 592.549164
roughness: 49.396259



For image 4 named 8F28F524-4ADC-4D6A-B212-CD01C180EC0F - Anil Chopra.jpeg:
Shape of image is: (1056, 969) 
coarseness: 9.433992
contrast: 25.780859
directionality: 533.010336
roughness: 35.214851



For image 5 named 3610B754-C9DD-47BD-BCEF-5757A1EBF5FA - A_JAY DHILOD.jpeg:
Shape of image is: (1295, 850) 
coarseness: 10.621658
contrast: 35.594186
directionality: 344.232013
roughness: 46.21584